# Collect shapes and generate relations

In this folder, we are building a dataset of geospatial shapes that have certain perscribed
relations, like "polygon intersection" or "point-on-line". 
The actual entities and their specific attributes don't matter. 

All shapes will be pulled from OpenStreetMap using the `osmnx` package. 
Since OSM is short on the Multi* type of entities, I will create them 
by combining random subsets of the other types.

We will not be encoding things in lon/lat space.
Instead we will focus on rectangular subsets of say 10km x 10 km. 
The mapping between lon/lat and local x/y will be done based on a local
transverse Mercator projection.


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx
import shapely
import pyproj

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter


In [ ]:
from geo_relations import ShapeHarvester
from geo_relations import RelationGenerator
from geo_relations import draw_shape

## Setup

In [ ]:
# Define a lon/lat center and a box size from which to pull shapes.
center_lat, center_lon = 42.631024, -70.993787 # Boxford MA
extent = 20000.0 # meters; this represents both width and height


In [ ]:
# Get a set of Point, LineString, and Polygon objects from our sample box.
harvester = ShapeHarvester(center_lon, center_lat, extent)
shapes = harvester.harvest(['points', 'linestrings', 'polygons'])
shapes['type'].value_counts()

In [ ]:
# Get some tiled polygons too. These will be used to generate 
# "polygon-borders-polygon" relations.
harvester = ShapeHarvester(center_lon, center_lat, extent * 2)
tiles = harvester.harvest(['tiled-polygons'])
tiles['type'].value_counts()

## Generate shape pairs with given relationships
For each type of relation, generate some positive and some negative cases, and plot them.

In [ ]:
aoi_width = 100
aoi_height = 100
cases_per = 3
relations = [
    'point-on-linestring', 'point-in-polygon', 'linestring-intersects-linestring',
    'linestring-intersects-polygon', 'polygon-intersects-polygon', 'polygon-borders-polygon'
]

for relation in relations:
    fodder = tiles if relation == 'polygon-borders-polygon' else shapes
    generator = RelationGenerator(fodder, bounds=[0, 0, aoi_width, aoi_height], scale=25)
    fig = make_subplots(2, cases_per, subplot_titles = ['true'] * cases_per + ['false'] * cases_per)
    
    for i in range(cases_per):
        a, b = generator.generate(relation, True, max_attempts=100)
        draw_shape(a, fig, irow=1, icol=i+1, name=a.geom_type, color='red')
        draw_shape(b, fig, irow=1, icol=i+1, name=b.geom_type, color='blue')
    
    for i in range(cases_per):
        a, b = generator.generate(relation, False)
        draw_shape(a, fig, irow=2, icol=i+1, name=a.geom_type, color='red')
        draw_shape(b, fig, irow=2, icol=i+1, name=b.geom_type, color='blue')
    
    fig['layout']['title'] = relation
    fig['layout']['width'] = 800
    fig['layout']['height'] = 600
    fig['layout']['showlegend'] = False
    
    for i in range(cases_per * 2):
        fig['layout']['xaxis%d' % (i+1)]['range'] = [0, aoi_width]
        fig['layout']['yaxis%d' % (i+1)]['range'] = [0, aoi_height]
    fig.show()


# fig.print_grid()